In [1]:
import json
import torch
from transformers import (
    DPRContextEncoder, DPRContextEncoderTokenizerFast,
    DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast,
    DPRReader, DPRReaderTokenizerFast, TrainingArguments, Trainer
)


/home/grads/r/rohan.chaudhury/miniconda3/envs/multidoc2dial/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-29 03:21:04.174684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 03:21:04.290495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-29 03:21:04.752224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.s

In [2]:
import transformers
print(transformers.__version__)

4.27.3


In [3]:
from transformers import DPRConfig, DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRReader, DPRReaderTokenizer
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

config = DPRConfig.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

reader = DPRReader.from_pretrained("facebook/dpr-reader-single-nq-base")
reader_tokenizer = DPRReaderTokenizer.from_pretrained("facebook/dpr-reader-single-nq-base")



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"  # Set this to the index of the GPU you want to use
import torch
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
question_encoder = nn.DataParallel(question_encoder)
context_encoder = nn.DataParallel(context_encoder)
question_encoder.to(device)
context_encoder.to(device)

DataParallel(
  (module): DPRContextEncoder(
    (ctx_encoder): DPREncoder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
            

In [ ]:
with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.train.json", "r") as f:
    training_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.psg.multidoc2dial_all.structure.json", "r") as f:
    corpus_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.validation.json", "r") as f:
    validation_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.test.json", "r") as f:
    test_data = json.load(f)


In [ ]:
import json
import re
from datasets import Dataset

def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def preprocess_question(question):
    turns = question.split("[SEP]")
    questions=turns[0]
    turns=[turns[1]]
    turns = [turn.strip() for turn in turns if turn.strip()]
    turns = [turn.split("||") for turn in turns]
    turns = [turn[::-1] for turn in turns]  # Reverse the order of previous turns
    turns = [" || ".join(turn) for turn in turns]

    return " ".join(turns).lower() + " || "+ "user: "+ questions.lower()



# # Preprocess the training dataset
# def preprocess_data(training_data):
#     train_data = {
#         "question": [],
#         "answers": [],
#         "ctx_title": [],
#         "ctx_text": [],
#         "label": []
#     }
    
#     for item in training_data:
#         question = preprocess_question(item["question"])
#         answers = [remove_extra_spaces(answer.lower()) for answer in item["answers"]]
#         positive_ctxs = item["positive_ctxs"]
#         negative_ctxs = item["negative_ctxs"]
#         hard_negative_ctxs = item["hard_negative_ctxs"]
        
#         for positive_ctx in positive_ctxs:
#             title = remove_extra_spaces(positive_ctx["title"].lower())
#             text = remove_extra_spaces(positive_ctx["text"].lower())
#             train_data["question"].append(question)
#             train_data["answers"].append(answers)
#             train_data["ctx_title"].append(title)
#             train_data["ctx_text"].append(text)
#             train_data["label"].append(1)
        
#         for negative_ctx in negative_ctxs:
#             title = remove_extra_spaces(negative_ctx["title"].lower())
#             text = remove_extra_spaces(negative_ctx["text"].lower())
#             train_data["question"].append(question)
#             train_data["answers"].append(answers)
#             train_data["ctx_title"].append(title)
#             train_data["ctx_text"].append(text)
#             train_data["label"].append(0)
        
#         for hard_negative_ctx in hard_negative_ctxs:
#             title = remove_extra_spaces(hard_negative_ctx["title"].lower())
#             text = remove_extra_spaces(hard_negative_ctx["text"].lower())
#             train_data["question"].append(question)
#             train_data["answers"].append(answers)
#             train_data["ctx_title"].append(title)
#             train_data["ctx_text"].append(text)
#             train_data["label"].append(-1)
    
#     return train_data

# def preprocess_corpus_data(corpus_data):
#     corpus_data_preprocessed = {
#         "title": [],
#         "text": []
#     }

#     for item in corpus_data:
#         title = remove_extra_spaces(item["title"].lower())
#         text = remove_extra_spaces(item["text"].lower())
#         corpus_data_preprocessed["title"].append(title)
#         corpus_data_preprocessed["text"].append(text)
    
#     return corpus_data_preprocessed

# corpus_data_dict = preprocess_corpus_data(corpus_data)
# corpus_dataset = Dataset.from_dict(corpus_data_dict)


# training_data= preprocess_data(training_data)
# test_data= preprocess_data(test_data)
# validation_data= preprocess_data(validation_data)

# train_dataset = Dataset.from_dict(training_data)
# test_dataset = Dataset.from_dict(test_data)
# validation_dataset = Dataset.from_dict(validation_data)



In [ ]:
def preprocess_data(training_data):
    train_data = {
        "question": [],
        "context": [],
        "labels": []
    }
    
    for item in training_data:
        question = preprocess_question(item["question"])
        positive_ctxs = item["positive_ctxs"]
        negative_ctxs = item["negative_ctxs"]
        hard_negative_ctxs = item["hard_negative_ctxs"]
        
        for positive_ctx in positive_ctxs:
            context = remove_extra_spaces(positive_ctx["title"].lower() + " " + positive_ctx["text"].lower())
            train_data["question"].append(question)
            train_data["context"].append(context)
            train_data["labels"].append(2)
        
        for negative_ctx in negative_ctxs:
            context = remove_extra_spaces(negative_ctx["title"].lower() + " " + negative_ctx["text"].lower())
            train_data["question"].append(question)
            train_data["context"].append(context)
            train_data["labels"].append(1)
        
        for hard_negative_ctx in hard_negative_ctxs:
            context = remove_extra_spaces(hard_negative_ctx["title"].lower() + " " + hard_negative_ctx["text"].lower())
            train_data["question"].append(question)
            train_data["context"].append(context)
            train_data["labels"].append(0)
    
    return train_data

def preprocess_corpus_data(corpus_data):
    corpus_data_preprocessed = {
        "title": [],
        "text": []
    }

    for item in corpus_data:
        title = remove_extra_spaces(item["title"].lower())
        text = remove_extra_spaces(item["text"].lower())
        corpus_data_preprocessed["title"].append(title)
        corpus_data_preprocessed["text"].append(text)
    
    return corpus_data_preprocessed

corpus_data_dict = preprocess_corpus_data(corpus_data)
corpus_dataset = Dataset.from_dict(corpus_data_dict)

# training_data= preprocess_data(training_data)
# test_data= preprocess_data(test_data)
# validation_data= preprocess_data(validation_data)

training_data= preprocess_data(training_data)
test_data= preprocess_data(test_data)
validation_data= preprocess_data(validation_data)

train_dataset = Dataset.from_dict(training_data)
test_dataset = Dataset.from_dict(test_data)
validation_dataset = Dataset.from_dict(validation_data)

# Add the preprocessing function that tokenizes the context and question
def preprocess_function(examples):
    question_encodings = question_tokenizer(examples['question'], truncation=True, padding='max_length', max_length=256)
    context_encodings = context_tokenizer(examples['context'], truncation=True, padding='max_length', max_length=256)
    
    encodings = {
        'input_ids': question_encodings['input_ids'],
        'attention_mask': question_encodings['attention_mask'],
        'context_input_ids': context_encodings['input_ids'],
        'context_attention_mask': context_encodings['attention_mask'],
        'labels': examples['labels'],
    }
    
    return encodings




In [ ]:
# train_dataset = train_dataset.map(preprocess_function, batched=True)
# validation_dataset = validation_dataset.map(preprocess_function, batched=True)

In [ ]:
# from torch.utils.data import Dataset as TorchDataset

# class CustomDataset(TorchDataset):
#     def __init__(self, data_dict):
#         self.data_dict = data_dict

#     def __getitem__(self, idx):
#         item = {key: value[idx] for key, value in self.data_dict.items()}
#         return preprocess_function(item)

#     def __len__(self):
#         return len(self.data_dict["attention_mask"])

# train_dataset = CustomDataset(training_data)
# validation_dataset = CustomDataset(validation_data)


In [ ]:
print(train_dataset[:2])


{'question': [' || user: hello, i forgot o update my address, can you help me with that?', ' || user: hello, i forgot o update my address, can you help me with that?'], 'context': ['top 5 dmv mistakes and how to avoid them#3_0 top 5 dmv mistakes and how to avoid them // 1. forgetting to update address by statute , you must report a change of address to dmv within ten days of moving. that is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. it is not sufficient to only: write your new address on the back of your old license; tell the united states postal service; or inform the police officer writing you a ticket. if you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors. this really happens , but the good news is this is a problem that is easily avoidable. learn m

In [ ]:
print(len(train_dataset[:]["labels"]))

450471


In [ ]:
# from transformers.data.data_collator import DataCollator


# class DPRDataCollator(DataCollator):
#     def __init__(self, question_tokenizer, context_tokenizer):
#         self.question_tokenizer = question_tokenizer
#         self.context_tokenizer = context_tokenizer

#     def __call__(self, batch):
#         question_texts = [item["question"] for item in batch]
#         context_texts = [item["context"] for item in batch]
#         labels = [item["label"] for item in batch]

#         question_inputs = self.question_tokenizer(question_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         context_inputs = self.context_tokenizer(context_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

#         batch_inputs = {
#             "input_ids": question_inputs["input_ids"],
#             "attention_mask": question_inputs["attention_mask"],
#             "context_input_ids": context_inputs["input_ids"],
#             "context_attention_mask": context_inputs["attention_mask"],
#             "labels": torch.tensor(labels, dtype=torch.long)
#         }

#         return batch_inputs


 training code does not use the corpus_data. The corpus_data is typically used during the retrieval step when you use the trained DPR model to find relevant context passages to answer a given question. The training step focuses on learning to differentiate between relevant and irrelevant context passages based on the labeled training data.

In [ ]:
# from torch import nn

# class DPRCombinedModel(nn.Module):
#     def __init__(self, question_encoder, context_encoder):
#         super(DPRCombinedModel, self).__init__()
#         self.question_encoder = question_encoder
#         self.context_encoder = context_encoder

#     def forward(self, input_ids, attention_mask, context_input_ids, context_attention_mask):
#         question_outputs = self.question_encoder(input_ids=input_ids, attention_mask=attention_mask)
#         context_outputs = self.context_encoder(input_ids=context_input_ids, attention_mask=context_attention_mask)
#         return question_outputs, context_outputs

import torch
import torch.nn as nn
from transformers import DPRQuestionEncoder, DPRContextEncoder

class DPRCombinedModel(nn.Module):
    def __init__(self, question_encoder: DPRQuestionEncoder, context_encoder: DPRContextEncoder):
        super(DPRCombinedModel, self).__init__()
        self.question_encoder = question_encoder
        self.context_encoder = context_encoder

    def forward(self, question_input_ids, question_attention_mask, context_input_ids, context_attention_mask):
        question_outputs = self.question_encoder(input_ids=question_input_ids, attention_mask=question_attention_mask)
        context_outputs = self.context_encoder(input_ids=context_input_ids, attention_mask=context_attention_mask)
        return question_outputs, context_outputs



In [ ]:
# from torch import nn
# from transformers import Trainer

# class DPRTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         print(inputs.keys())
#         input_ids = inputs["input_ids"]
#         attention_mask = inputs["attention_mask"]
#         context_input_ids = inputs["context_input_ids"]
#         context_attention_mask = inputs["context_attention_mask"]
#         labels = inputs["labels"]

#         question_outputs, context_outputs = model(input_ids, attention_mask, context_input_ids, context_attention_mask)

#         # Calculate dot product between question and context embeddings
#         scores = torch.matmul(question_outputs.pooler_output, context_outputs.pooler_output.T)

#         # Calculate the loss
#         loss_fct = nn.CrossEntropyLoss()
#         loss = loss_fct(scores, labels)

#         return (loss, question_outputs, context_outputs) if return_outputs else loss


In [ ]:
# from typing import List, Dict, Tuple, Union
# import torch

# class DPRDataCollator:
#     def __init__(self, question_tokenizer, context_tokenizer, max_length: int):
#         self.question_tokenizer = question_tokenizer
#         self.context_tokenizer = context_tokenizer
#         self.max_length = max_length
    
#     def __call__(self, features):
#         print(features[0].keys())
#         input_ids = torch.tensor([f["input_ids"] for f in features], dtype=torch.long)
#         attention_mask = torch.tensor([f["attention_mask"] for f in features], dtype=torch.long)
#         context_input_ids = torch.tensor([f["context_input_ids"] for f in features], dtype=torch.long)
#         context_attention_mask = torch.tensor([f["context_attention_mask"] for f in features], dtype=torch.long)
#         labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)

#         return {
#             "input_ids": input_ids,
#             "attention_mask": attention_mask,
#             "context_input_ids": context_input_ids,
#             "context_attention_mask": context_attention_mask,
#             "labels": labels,
#         }

In [ ]:
# from transformers import TrainingArguments
# from transformers.data.data_collator import DataCollator


# def train_dpr_model(train_dataset, gradient_accumulation_steps, per_device_train_batch_size, num_train_epochs, output_dir):
#     combined_model = DPRCombinedModel(question_encoder, context_encoder)
    
#     training_args = TrainingArguments(
#         gradient_accumulation_steps=gradient_accumulation_steps,
#         per_device_train_batch_size=per_device_train_batch_size,
#         num_train_epochs=num_train_epochs,
#         output_dir=output_dir,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="exact_match",
#         save_total_limit=1,
#     )

#     data_collator = DPRDataCollator(question_tokenizer, context_tokenizer, max_length=512)

#     trainer = DPRTrainer(
#         model=combined_model,
#         args=training_args,
#         train_dataset=train_dataset,
#         data_collator=data_collator,
#         eval_dataset=validation_dataset,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
#     )

#     trainer.train()

# print(train_dataset[:2].keys())
# train_dpr_model(train_dataset, gradient_accumulation_steps=4, per_device_train_batch_size=4, num_train_epochs=3, output_dir="./dpr_output")


In [ ]:
# import torch
# from torch.optim import AdamW
# from torch.utils.data import DataLoader
# from transformers import get_linear_schedule_with_warmup
# import torch.nn.functional as F

# from math import ceil
# def train_dpr_model(train_dataset, validation_dataset, model, question_tokenizer, context_tokenizer, epochs, batch_size, learning_rates, device, gradient_accumulation_steps=1, num_warmup_steps=0):
#     # train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=DPRDataCollator(question_tokenizer, context_tokenizer, max_length=512))
#     # val_dataloader = DataLoader(validation_dataset, batch_size=batch_size, collate_fn=DPRDataCollator(question_tokenizer, context_tokenizer, max_length=512))

#     question_encoder_optimizer = AdamW(model.question_encoder.parameters(), lr=learning_rates['question_encoder'])
#     context_encoder_optimizer = AdamW(model.context_encoder.parameters(), lr=learning_rates['context_encoder'])
#     number_of_batches = ceil(len(train_dataset) / batch_size)
#     total_steps = number_of_batches * epochs // gradient_accumulation_steps
#     question_encoder_scheduler = get_linear_schedule_with_warmup(question_encoder_optimizer, num_warmup_steps, total_steps)
#     context_encoder_scheduler = get_linear_schedule_with_warmup(context_encoder_optimizer, num_warmup_steps, total_steps)

#     model.to(device)
#     best_val_loss = float('inf')
#     best_model = None
#     max_length = 512
#     for epoch in range(epochs):
#         print(f"Epoch {epoch + 1}/{epochs}")
#         model.train()
#         total_loss = 0.0

#         # for idx, batch in enumerate(train_dataloader):
#         #     input_ids = batch["input_ids"].to(device)
#         #     attention_mask = batch["attention_mask"].to(device)
#         #     context_input_ids = batch["context_input_ids"].to(device)
#         #     context_attention_mask = batch["context_attention_mask"].to(device)
#         #     labels = batch["labels"].to(device)

#         for i in range(0, len(train_dataset), batch_size):
#             # raw_batch = train_dataset[i:i + batch_size]

#             # Tokenize questions and contexts within the loop
#             # question_encodings = question_tokenizer([example["question"] for example in raw_batch], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             # context_encodings = context_tokenizer([example["context"] for example in raw_batch], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             # labels = torch.tensor([example["label"] for example in raw_batch], dtype=torch.long).to(device)

#             question_encodings = question_tokenizer([train_dataset[example_idx]["question"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             context_encodings = context_tokenizer([train_dataset[example_idx]["context"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             labels = torch.tensor([train_dataset[example_idx]["labels"] for example_idx in range(i, i + batch_size)], dtype=torch.long).to(device)


#             input_ids = question_encodings['input_ids'].to(device)
#             attention_mask = question_encodings['attention_mask'].to(device)
#             context_input_ids = context_encodings['input_ids'].to(device)
#             context_attention_mask = context_encodings['attention_mask'].to(device)
#             question_outputs, context_outputs = model(input_ids, attention_mask, context_input_ids, context_attention_mask)

#             scores = torch.matmul(question_outputs.pooler_output, context_outputs.pooler_output.T)

#             one_hot_labels = F.one_hot(labels, num_classes=scores.size(1))
#             loss = F.binary_cross_entropy_with_logits(scores, one_hot_labels.float())

#             total_loss += loss.item()

#             loss = loss / gradient_accumulation_steps
#             loss.backward()

#             if (i + 1) % gradient_accumulation_steps == 0:
#                 question_encoder_optimizer.step()
#                 context_encoder_optimizer.step()
#                 question_encoder_scheduler.step()
#                 context_encoder_scheduler.step()

#                 model.zero_grad()

#         avg_train_loss = total_loss / number_of_batches
#         print(f"Training loss: {avg_train_loss}")

#         model.eval()
#         total_val_loss = 0.0
#         number_of_batches_validation = ceil(len(validation_dataset) / batch_size)
#         for i in range(0, len(validation_dataset), batch_size):

#             # Tokenize questions and contexts within the loop
#             # question_encodings = question_tokenizer([example["question"] for example in raw_batch], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             # context_encodings = context_tokenizer([example["context"] for example in raw_batch], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             # labels = torch.tensor([example["label"] for example in raw_batch], dtype=torch.long).to(device)

#             question_encodings = question_tokenizer([validation_dataset[example_idx]["question"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             context_encodings = context_tokenizer([validation_dataset[example_idx]["context"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
#             labels = torch.tensor([validation_dataset[example_idx]["labels"] for example_idx in range(i, i + batch_size)], dtype=torch.long).to(device)


#             input_ids = question_encodings['input_ids'].to(device)
#             attention_mask = question_encodings['attention_mask'].to(device)
#             context_input_ids = context_encodings['input_ids'].to(device)
#             context_attention_mask = context_encodings['attention_mask'].to(device)

#             with torch.no_grad():
#                 question_outputs, context_outputs = model(input_ids, attention_mask, context_input_ids, context_attention_mask)
#                 scores = torch.matmul(question_outputs.pooler_output, context_outputs.pooler_output.T)
#                 one_hot_labels = F.one_hot(labels, num_classes=scores.size(1))
#                 loss = F.binary_cross_entropy_with_logits(scores, one_hot_labels.float())

#                 total_val_loss += loss.item()

#         avg_val_loss = total_val_loss / number_of_batches_validation
#         print(f"Validation loss: {avg_val_loss}")

#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             best_model = DPRCombinedModel(model.question_encoder, model.context_encoder)
#             best_model.load_state_dict(model.state_dict())
#             torch.save(best_model.state_dict(), "best_dpr_model.pth")
#             print(f"Best model saved with validation loss: {best_val_loss}")

#     print("Training complete.")
#     return best_model

# # Train the model
# epochs = 3
# batch_size = 4
# learning_rates = {
#     'question_encoder': 5e-5,
#     'context_encoder': 5e-5
# }
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# gradient_accumulation_steps = 4
# num_warmup_steps = 0

# combined_model = DPRCombinedModel(question_encoder, context_encoder)
# best_model = train_dpr_model(train_dataset, validation_dataset, combined_model, question_tokenizer, context_tokenizer, epochs, batch_size, learning_rates, device, gradient_accumulation_steps, num_warmup_steps)


In [ ]:
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from math import ceil
from tqdm import tqdm

def train_dpr_model(train_dataset, validation_dataset, model, question_tokenizer, context_tokenizer, epochs, batch_size, learning_rates, device, gradient_accumulation_steps=1, num_warmup_steps=0):

    question_encoder_optimizer = AdamW(model.question_encoder.parameters(), lr=learning_rates['question_encoder'])
    context_encoder_optimizer = AdamW(model.context_encoder.parameters(), lr=learning_rates['context_encoder'])
    number_of_batches = ceil(len(train_dataset) / batch_size)
    total_steps = number_of_batches * epochs // gradient_accumulation_steps
    question_encoder_scheduler = get_linear_schedule_with_warmup(question_encoder_optimizer, num_warmup_steps, total_steps)
    context_encoder_scheduler = get_linear_schedule_with_warmup(context_encoder_optimizer, num_warmup_steps, total_steps)

    model.to(device)
    best_val_loss = float('inf')
    best_model = None
    max_length = 512
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0.0
        train_iter = tqdm(range(0, len(train_dataset), batch_size), desc="Training", ncols=100)

        for i in train_iter:

            question_encodings = question_tokenizer([train_dataset[example_idx]["question"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
            context_encodings = context_tokenizer([train_dataset[example_idx]["context"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
            labels = torch.tensor([train_dataset[example_idx]["labels"] for example_idx in range(i, i + batch_size)], dtype=torch.long).to(device)

            input_ids = question_encodings['input_ids'].to(device)
            attention_mask = question_encodings['attention_mask'].to(device)
            context_input_ids = context_encodings['input_ids'].to(device)
            context_attention_mask = context_encodings['attention_mask'].to(device)
            question_outputs, context_outputs = model(input_ids, attention_mask, context_input_ids, context_attention_mask)

            scores = torch.matmul(question_outputs.pooler_output, context_outputs.pooler_output.T)

            one_hot_labels = F.one_hot(labels, num_classes=scores.size(1))
            loss = F.binary_cross_entropy_with_logits(scores, one_hot_labels.float())
            total_loss += loss.item()

            loss = loss / gradient_accumulation_steps
            loss.backward()

            if (i + 1) % gradient_accumulation_steps == 0:
                question_encoder_optimizer.step()
                context_encoder_optimizer.step()
                question_encoder_scheduler.step()
                context_encoder_scheduler.step()

                model.zero_grad()
            
            train_iter.set_description(f"Training (loss = {loss.item():.4f})")
            train_iter.refresh()

        avg_train_loss = total_loss / number_of_batches
        print(f"Training loss: {avg_train_loss}")

        model.eval()
        total_val_loss = 0.0
        number_of_batches_validation = ceil(len(validation_dataset) / batch_size)
        val_iter = tqdm(range(0, len(validation_dataset), batch_size), desc="Validation", ncols=100)
        for i in val_iter:

            question_encodings = question_tokenizer([validation_dataset[example_idx]["question"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
            context_encodings = context_tokenizer([validation_dataset[example_idx]["context"] for example_idx in range(i, i + batch_size)], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
            labels = torch.tensor([validation_dataset[example_idx]["labels"] for example_idx in range(i, i + batch_size)], dtype=torch.long).to(device)

            input_ids = question_encodings['input_ids'].to(device)
            attention_mask = question_encodings['attention_mask'].to(device)
            context_input_ids = context_encodings['input_ids'].to(device)
            context_attention_mask = context_encodings['attention_mask'].to(device)

            with torch.no_grad():
                question_outputs, context_outputs = model(input_ids, attention_mask, context_input_ids, context_attention_mask)
                scores = torch.matmul(question_outputs.pooler_output, context_outputs.pooler_output.T)
                one_hot_labels = F.one_hot(labels, num_classes=scores.size(1))
                loss = F.binary_cross_entropy_with_logits(scores, one_hot_labels.float())
                total_val_loss += loss.item()
            val_iter.set_description(f"Validation (loss = {loss.item():.4f})")
            val_iter.refresh()
            
        avg_val_loss = total_val_loss / number_of_batches_validation
        print(f"Validation loss: {avg_val_loss}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = DPRCombinedModel(model.question_encoder, model.context_encoder)
            best_model.load_state_dict(model.state_dict())
            torch.save(best_model.state_dict(), "best_dpr_model.pth")
            print(f"Best model saved with validation loss: {best_val_loss}")

    print("Training complete.")
    return best_model



In [ ]:
epochs = 3
batch_size = 8
learning_rates = {
'question_encoder': 5e-5,
'context_encoder': 5e-5
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gradient_accumulation_steps = 8
num_warmup_steps = 0

combined_model = DPRCombinedModel(question_encoder, context_encoder)
best_model = train_dpr_model(train_dataset, validation_dataset, combined_model, question_tokenizer, context_tokenizer, epochs, batch_size, learning_rates, device, gradient_accumulation_steps, num_warmup_steps)

Epoch 1/3


Training (loss = 7.4187):   0%|                               | 1/56309 [00:30<470:20:48, 30.07s/it]../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [3,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [4,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [5,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [6,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [7,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bo

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
# from transformers import TrainingArguments

# def train_dpr_model(train_dataset, gradient_accumulation_steps, per_device_train_batch_size, num_train_epochs, output_dir):
#     combined_model = DPRCombinedModel(question_encoder, context_encoder)
    
#     training_args = TrainingArguments(
#         gradient_accumulation_steps=gradient_accumulation_steps,
#         per_device_train_batch_size=per_device_train_batch_size,
#         num_train_epochs=num_train_epochs,
#         output_dir=output_dir,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="exact_match",
#         save_total_limit=1,
#     )

#     trainer = DPRTrainer(
#         model=combined_model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=validation_dataset,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
#     )

#     trainer.train()

# train_dpr_model(train_dataset, gradient_accumulation_steps=4, per_device_train_batch_size=4, num_train_epochs=3, output_dir="./dpr_output")


In [ ]:


# from transformers import Trainer

# class DPRTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss_fct = torch.nn.CrossEntropyLoss()
#         loss = loss_fct(logits, labels)
        
#         return (loss, outputs) if return_outputs else loss



In [ ]:
# def preprocess_function(examples):
#     question_encodings = question_tokenizer(examples['question'], truncation=True, padding='max_length', max_length=64)
#     context_encodings = context_tokenizer(examples['context'], truncation=True, padding='max_length', max_length=256)
    
#     encodings = {
#         'input_ids': question_encodings['input_ids'],
#         'attention_mask': question_encodings['attention_mask'],
#         'context_input_ids': context_encodings['input_ids'],
#         'context_attention_mask': context_encodings['attention_mask'],
#         'labels': examples['labels'],
#     }
    
#     return encodings

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# validation_dataset = validation_dataset.map(preprocess_function, batched=True)

In [ ]:
# def train_dpr_model(train_dataset, gradient_accumulation_steps, per_device_train_batch_size, num_train_epochs, output_dir):
#     training_args = TrainingArguments(
#         gradient_accumulation_steps=gradient_accumulation_steps,
#         per_device_train_batch_size=per_device_train_batch_size,
#         num_train_epochs=num_train_epochs,
#         output_dir=output_dir,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="exact_match",
#         save_total_limit=1,
#     )

#     trainer = DPRTrainer(
#         model=reader,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=validation_dataset,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
#     )

#     trainer.train()

# train_dpr_model(train_dataset, gradient_accumulation_steps=4, per_device_train_batch_size=4, num_train_epochs=3, output_dir="./dpr_output")
